In [50]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
#from qiskit_aer.library import save_statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import numpy as np
from qiskit.quantum_info import Statevector

In [51]:
np.random.seed(42)

# Defining relevant methods for shor's code
Note that this implementation, does not use ancillary bits to detect which qubit is bit/phase flipped, i.e without stabilizer measurement (8 qubits saved), but rather just detects and correct the main qubit.

In [52]:
def bit_flip_encode(qc,qubits):
    "encodes the bit flip code, 2nd 3rd index of qubits are the added physical qubits"
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[0], qubits[2])
def phase_flip_encode(qc,qubits):
    "encodes the phase flip code, 2nd 3rd index of qubits are the added physical qubits"
    bit_flip_encode(qc,qubits)
    qc.h(qubits)
def bit_flip_detect_correct(qc,qubits):
    "detects and corrects the bit flip on first qubit"
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[0], qubits[2])
    qc.ccx(qubits[1], qubits[2], qubits[0])

def phase_flip_detect_correct(qc,qubits):
    "detects and corrects the phase flip on first qubit"
    qc.h(qubits)
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[0], qubits[2])
    qc.ccx(qubits[1], qubits[2], qubits[0])

# Shor's code to protect from bit and phase flip errors
It is a combination of bit flip and phase flip detector and correctors circuits used in part A.

In [53]:
# Bit flip code
qr1 = QuantumRegister(9, 'q')
qc1 = QuantumCircuit(qr1)

#preparing random qubit state
qc1.rx(np.pi/np.random.randint(2,10),qr1[0])
qc1.ry(np.pi/np.random.randint(2,10),qr1[0])
qc1.rz(np.pi/np.random.randint(2,10),qr1[0])

# encoding for phase flip detection and correction
qc1.save_statevector(label="initial state")
phase_flip_encode(qc1,qr1[::3])
qc1.save_statevector(label="after phase encoding")

# further encoding for bit flip detection and correction (at each three consecutive qubits blocks)
bit_flip_encode(qc1,qr1[:3])
bit_flip_encode(qc1,qr1[3:6])
bit_flip_encode(qc1,qr1[6:9])
qc1.save_statevector(label="after bit encoding")

# introducing error on the anyone qubit
qc1.y(qr1[np.random.randint(0,3)])
#qc1.z(qr1[np.random.randint(3,6)])
#qc1.x(qr1[np.random.randint(6,9)])
qc1.save_statevector(label="after error")

# bit flip detection and correction
bit_flip_detect_correct(qc1,qr1[:3])
bit_flip_detect_correct(qc1,qr1[3:6])
bit_flip_detect_correct(qc1,qr1[6:9])
qc1.save_statevector(label="after bit flip correction")

# phase flip detection and correction
phase_flip_detect_correct(qc1,qr1[::3])
qc1.save_statevector(label="after phase flip correction")

qc1.draw("text")

┌─────────┐┌─────────┐┌─────────┐ initial state           ┌───┐»
q_0: ┤ Rx(π/8) ├┤ Ry(π/5) ├┤ Rz(π/6) ├───────░─────────■────■──┤ H ├»
     └─────────┘└─────────┘└─────────┘       ░         │    │  └───┘»
q_1: ────────────────────────────────────────░─────────┼────┼───────»
                                             ░         │    │       »
q_2: ────────────────────────────────────────░─────────┼────┼───────»
                                             ░       ┌─┴─┐  │  ┌───┐»
q_3: ────────────────────────────────────────░───────┤ X ├──┼──┤ H ├»
                                             ░       └───┘  │  └───┘»
q_4: ────────────────────────────────────────░──────────────┼───────»
                                             ░              │       »
q_5: ────────────────────────────────────────░──────────────┼───────»
                                             ░            ┌─┴─┐┌───┐»
q_6: ────────────────────────────────────────░────────────┤ X ├┤ H ├»
                                             ░            └───┘└───┘»
q_7: ────────────────────────────────────────░──────────────────────»
                                             ░                      »
q_8: ────────────────────────────────────────░──────────────────────»
                                             ░                      »
«      after phase encoding            after bit encoding       after error »
«q_0: ──────────░─────────────■────■───────────░─────────────────────░──────»
«               ░           ┌─┴─┐  │           ░                     ░      »
«q_1: ──────────░───────────┤ X ├──┼───────────░─────────────────────░──────»
«               ░           └───┘┌─┴─┐         ░          ┌───┐      ░      »
«q_2: ──────────░────────────────┤ X ├─────────░──────────┤ Y ├──────░──────»
«               ░                └───┘         ░          └───┘      ░      »
«q_3: ──────────░─────────────■────■───────────░─────────────────────░──────»
«               ░           ┌─┴─┐  │           ░                     ░      »
«q_4: ──────────░───────────┤ X ├──┼───────────░─────────────────────░──────»
«               ░           └───┘┌─┴─┐         ░                     ░      »
«q_5: ──────────░────────────────┤ X ├─────────░─────────────────────░──────»
«               ░                └───┘         ░                     ░      »
«q_6: ──────────░─────────────■────■───────────░─────────────────────░──────»
«               ░           ┌─┴─┐  │           ░                     ░      »
«q_7: ──────────░───────────┤ X ├──┼───────────░─────────────────────░──────»
«               ░           └───┘┌─┴─┐         ░                     ░      »
«q_8: ──────────░────────────────┤ X ├─────────░─────────────────────░──────»
«               ░                └───┘         ░                     ░      »
«               ┌───┐ after bit flip correction ┌───┐          ┌───┐»
«q_0: ──■────■──┤ X ├─────────────░─────────────┤ H ├──■────■──┤ X ├»
«     ┌─┴─┐  │  └─┬─┘             ░             └───┘  │    │  └─┬─┘»
«q_1: ┤ X ├──┼────■───────────────░────────────────────┼────┼────┼──»
«     └───┘┌─┴─┐  │               ░                    │    │    │  »
«q_2: ─────┤ X ├──■───────────────░────────────────────┼────┼────┼──»
«          └───┘┌───┐             ░             ┌───┐┌─┴─┐  │    │  »
«q_3: ──■────■──┤ X ├─────────────░─────────────┤ H ├┤ X ├──┼────■──»
«     ┌─┴─┐  │  └─┬─┘             ░             └───┘└───┘  │    │  »
«q_4: ┤ X ├──┼────■───────────────░─────────────────────────┼────┼──»
«     └───┘┌─┴─┐  │               ░                         │    │  »
«q_5: ─────┤ X ├──■───────────────░─────────────────────────┼────┼──»
«          └───┘┌───┐             ░             ┌───┐     ┌─┴─┐  │  »
«q_6: ──■────■──┤ X ├─────────────░─────────────┤ H ├─────┤ X ├──■──»
«     ┌─┴─┐  │  └─┬─┘             ░             └───┘     └───┘     »
«q_7: ┤ X ├──┼────■───────────────░─────────────────────────────────»
«     └───┘┌─┴─┐  │               ░                                 »
«q_

In [54]:
sim = AerSimulator(method='statevector')
pm = generate_preset_pass_manager(backend=sim)
qc1 = pm.run(qc1)
res1 = sim.run(qc1).result().data()

In [55]:
res1["initial state"].draw(output="latex")   # main qubit is rightmost

<IPython.core.display.Latex object>

In [56]:
res1["after phase encoding"].draw(output="latex")

<IPython.core.display.Latex object>

In [57]:
res1["after bit encoding"].draw(output="latex")

<IPython.core.display.Latex object>

In [58]:
res1["after error"].draw(output="latex")

<IPython.core.display.Latex object>

In [59]:
res1["after bit flip correction"].draw(output="latex")

<IPython.core.display.Latex object>

In [60]:
res1["after phase flip correction"].draw(output="latex")

<IPython.core.display.Latex object>

# CSS code to protect from bit and phase flips
The CSS Code encodes 1 logical qubit into 7 physical qubits using the [7,4,3] Hamming code structure

## Required methods

In [61]:
def encode_css(qc, qubits):
    " encodes to css code, where first 7 qubits are the encoding qubits and rest are ancillas"
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[0], qubits[2])
    qc.cx(qubits[0], qubits[3])
    qc.cx(qubits[1], qubits[4])
    qc.cx(qubits[2], qubits[5])
    qc.cx(qubits[3], qubits[6])

def decode_css(qc, data_qubits):
    # Reverse the encoding steps
    qc.cx(data_qubits[3], data_qubits[6])
    qc.cx(data_qubits[2], data_qubits[5])
    qc.cx(data_qubits[1], data_qubits[4])
    qc.cx(data_qubits[0], data_qubits[3])
    qc.cx(data_qubits[0], data_qubits[2])
    qc.cx(data_qubits[0], data_qubits[1])

def measure_stabilizers(qc, data_qubits, ancilla_qubits, syndrome_bits):
    # X stabilizers (detect phase flips)
    x_stabilizers = [
        [0, 1, 2, 4],  # X₀X₁X₂X₄
        [0, 1, 3, 5],  # X₀X₁X₃X₅
        [0, 2, 3, 6]   # X₀X₂X₃X₆
    ]
    # Z stabilizers (detect bit flips)
    z_stabilizers = [
        [0, 1, 2, 4],  # Z₀Z₁Z₂Z₄
        [0, 1, 3, 5],  # Z₀Z₁Z₃Z₅
        [0, 2, 3, 6]   # Z₀Z₂Z₃Z₆
    ]
    
    # Measure X stabilizers
    for i, stab in enumerate(x_stabilizers):
        qc.reset(ancilla_qubits[i])
        for qubit in stab:
            qc.cx(ancilla_qubits[i], data_qubits[qubit])
        qc.h(ancilla_qubits[i])
        qc.measure(ancilla_qubits[i], syndrome_bits[i])
    
    # Measure Z stabilizers
    for i, stab in enumerate(z_stabilizers):
        qc.reset(ancilla_qubits[i+3])
        for qubit in stab:
            qc.cz(ancilla_qubits[i+3], data_qubits[qubit])
        qc.measure(ancilla_qubits[i+3], syndrome_bits[i+3])


syndrome_table = {
    # Format: 's₀s₁s₂s₃s₄s₅' : (error_type, qubit_index)
    '000000': ('None', None),
    '111000': ('Z', 0),  # Phase-flip on qubit 0
    '110000': ('Z', 1),  # Phase-flip on qubit 1
    '101000': ('Z', 2),  # Phase-flip on qubit 2
    '011000': ('Z', 3),  # Phase-flip on qubit 3
    '100000': ('Z', 4),  # Phase-flip on qubit 4
    '010000': ('Z', 5),  # Phase-flip on qubit 5
    '001000': ('Z', 6),  # Phase-flip on qubit 6
    '000111': ('X', 0),  # Bit-flip on qubit 0
    '000110': ('X', 1),  # Bit-flip on qubit 1
    '000101': ('X', 2),  # Bit-flip on qubit 2
    '000011': ('X', 3),  # Bit-flip on qubit 3
    '000100': ('X', 4),  # Bit-flip on qubit 4
    '000010': ('X', 5),  # Bit-flip on qubit 5
    '000001': ('X', 6),  # Bit-flip on qubit 6
}

def correct_errors(qc,data_qubits, syndrome_bits):
    qc.z(data_qubits[0]).c_if(syndrome_bits, 7)
    qc.z(data_qubits[1]).c_if(syndrome_bits, 3)
    qc.z(data_qubits[2]).c_if(syndrome_bits, 5)
    qc.z(data_qubits[3]).c_if(syndrome_bits, 6)
    qc.z(data_qubits[4]).c_if(syndrome_bits, 1)
    qc.z(data_qubits[5]).c_if(syndrome_bits, 2)
    qc.z(data_qubits[6]).c_if(syndrome_bits, 4)

    qc.x(data_qubits[0]).c_if(syndrome_bits, 56)
    qc.x(data_qubits[1]).c_if(syndrome_bits, 24)
    qc.x(data_qubits[2]).c_if(syndrome_bits, 40)
    qc.x(data_qubits[3]).c_if(syndrome_bits, 48)
    qc.x(data_qubits[4]).c_if(syndrome_bits, 8)
    qc.x(data_qubits[5]).c_if(syndrome_bits, 16)
    qc.x(data_qubits[6]).c_if(syndrome_bits, 32)


In [62]:
qr2 = QuantumRegister(13, 'q')
cr2 = ClassicalRegister(6, 'c')
qc2 = QuantumCircuit(qr2,cr2)

# preparing random qubit state
qc2.rx(np.pi/np.random.randint(2,10),qr2[0])
qc2.ry(np.pi/np.random.randint(2,10),qr2[0])
qc2.rz(np.pi/np.random.randint(2,10),qr2[0])
qc2.save_statevector(label="initial state")

# encoding css code
encode_css(qc2,qr2)
qc2.save_statevector(label="after encoding")

# introducing error( bit and phase flip ) on the anyone qubit
qc2.y(qr2[np.random.randint(0,7)])
qc2.save_statevector(label="after error")

# measuring stabilizers
measure_stabilizers(qc2, qr2[:7], qr2[7:], cr2)
qc2.save_statevector(label="after stabilizer measurement")

# decoding css code
decode_css(qc2, qr2[:7])
qc2.save_statevector(label="after decoding")

# correcting errors
correct_errors(qc2, qr2[:7], cr2)
qc2.save_statevector(label="after correction")
qc2.draw("text")



/tmp/ipykernel_8672/3476384616.py:69: DeprecationWarning: The method ``qiskit.circuit.instructionset.InstructionSet.c_if()`` is deprecated as of qiskit 1.3.0. It will be removed in 2.0.0.
  qc.z(data_qubits[0]).c_if(syndrome_bits, 7)
/tmp/ipykernel_8672/3476384616.py:70: DeprecationWarning: The method ``qiskit.circuit.instructionset.InstructionSet.c_if()`` is deprecated as of qiskit 1.3.0. It will be removed in 2.0.0.
  qc.z(data_qubits[1]).c_if(syndrome_bits, 3)
/tmp/ipykernel_8672/3476384616.py:71: DeprecationWarning: The method ``qiskit.circuit.instructionset.InstructionSet.c_if()`` is deprecated as of qiskit 1.3.0. It will be removed in 2.0.0.
  qc.z(data_qubits[2]).c_if(syndrome_bits, 5)
/tmp/ipykernel_8672/3476384616.py:72: DeprecationWarning: The method ``qiskit.circuit.instructionset.InstructionSet.c_if()`` is deprecated as of qiskit 1.3.0. It will be removed in 2.0.0.
  qc.z(data_qubits[3]).c_if(syndrome_bits, 6)
/tmp/ipykernel_8672/3476384616.py:73: DeprecationWarning: The me

┌─────────┐┌─────────┐┌─────────┐ initial state                          »
 q_0: ┤ Rx(π/4) ├┤ Ry(π/9) ├┤ Rz(π/6) ├───────░─────────■────■─────────■───────»
      └─────────┘└─────────┘└─────────┘       ░       ┌─┴─┐  │         │       »
 q_1: ────────────────────────────────────────░───────┤ X ├──┼────■────┼───────»
                                              ░       └───┘┌─┴─┐  │    │       »
 q_2: ────────────────────────────────────────░────────────┤ X ├──┼────┼────■──»
                                              ░            └───┘  │  ┌─┴─┐  │  »
 q_3: ────────────────────────────────────────░───────────────────┼──┤ X ├──┼──»
                                              ░                 ┌─┴─┐└───┘  │  »
 q_4: ────────────────────────────────────────░─────────────────┤ X ├───────┼──»
                                              ░                 └───┘     ┌─┴─┐»
 q_5: ────────────────────────────────────────░───────────────────────────┤ X ├»
                                              ░                           └───┘»
 q_6: ────────────────────────────────────────░────────────────────────────────»
                                              ░                                »
 q_7: ────────────────────────────────────────░────────────────────────────────»
                                              ░                                »
 q_8: ────────────────────────────────────────░────────────────────────────────»
                                              ░                                »
 q_9: ────────────────────────────────────────░────────────────────────────────»
                                              ░                                »
q_10: ────────────────────────────────────────░────────────────────────────────»
                                              ░                                »
q_11: ────────────────────────────────────────░────────────────────────────────»
                                              ░                                »
q_12: ────────────────────────────────────────░────────────────────────────────»
                                              ░                                »
 c: 6/═════════════════════════════════════════════════════════════════════════»
                                                                               »
«            after encoding       after error      ┌───┐     ┌───┐          »
« q_0: ────────────░───────────────────░───────────┤ X ├─────┤ X ├──────────»
«                  ░                   ░           └─┬─┘┌───┐└─┬─┘     ┌───┐»
« q_1: ────────────░───────────────────░─────────────┼──┤ X ├──┼───────┤ X ├»
«                  ░                   ░             │  └─┬─┘  │  ┌───┐└─┬─┘»
« q_2: ────────────░───────────────────░─────────────┼────┼────┼──┤ X ├──┼──»
«                  ░                   ░             │    │    │  └─┬─┘  │  »
« q_3: ──■─────────░───────────────────░─────────────┼────┼────┼────┼────┼──»
«        │         ░        ┌───┐      ░             │    │    │    │    │  »
« q_4: ──┼─────────░────────┤ Y ├──────░─────────────┼────┼────┼────┼────┼──»
«        │         ░        └───┘      ░             │    │    │    │    │  »
« q_5: ──┼─────────░───────────────────░─────────────┼────┼────┼────┼────┼──»
«      ┌─┴─┐       ░                   ░             │    │    │    │    │  »
« q_6: ┤ X ├───────░───────────────────░─────────────┼────┼────┼────┼────┼──»
«      └───┘       ░                   ░             │    │    │    │    │  »
« q_7: ────────────░───────────────────░───────|0>───■────■────┼────■────┼──»
«                  ░                   ░                       │         │  »
« q_8: ────────────░───────────────────░───────|0>─────────────■─────────■──»
«                  ░                   ░                                    »
« q_9: ────────────░───────────────────░───────|0>──────────────────────────»
«                  ░                   ░                                    »
«q_10: ────────────

In [63]:
sim = AerSimulator(method='statevector')
pm = generate_preset_pass_manager(backend=sim)
qc2 = pm.run(qc2)
res2 = sim.run(qc2).result().data()

In [64]:
res2["initial state"].draw(output="latex")   # main qubit is rightmost

<IPython.core.display.Latex object>

In [65]:
res2["after encoding"].draw(output="latex")

<IPython.core.display.Latex object>

In [66]:
res2["after error"].draw(output="latex")

<IPython.core.display.Latex object>

In [67]:
res2["after stabilizer measurement"].draw(output="latex")

<IPython.core.display.Latex object>

In [68]:
res2["after decoding"].draw(output="latex")

<IPython.core.display.Latex object>

In [69]:
res2["after correction"].draw(output="latex")

<IPython.core.display.Latex object>

# Answers to some theoretical questions
## 1: What is a stabilizer code, and how does it relate to the Shor and Steane codes?
> - ng Pauli operators to define valid code states. A state $\ket\psi_L$ is valid if $S_i \ket\psi_L = + 1 \ket\psi_L$ for all >stabilizers $S_i$.
> - Shor Code: Combines inner 3-qubit phase-flip and outer 3-qubit bit-flip codes. Stabilizers include $Z_i Z_j$ (bit flip >detection) and $X_i X_j X_k X_l$ (phase flip detection).
> - Steane Code: A CSS code derived from classical [7,4,3] Hamming code. Stabilizers split into X-type (phase-flip detection) and >Z-type (bit-flip detection).

## 2: Compare the Shor Code and the Steane Code. What are the advantages and disadvantages of each?


| Feature | Shor code (9 qubits) | CSS code (7 qubits) |
| :----------------: | :--------------------: | :-------------------: | 
| Error correction | Corrects arbitrary single-qubit errors | Corrects X/Z errors separately (CSS structure) |
| Qubit overhead   | 	High (9 qubits) | Moderate (7 qubits) |
| Stabilizers   | Simple (3 Z-type, 2 X-type)   | Complex (6 stabilizers: 3 X, 3 Z) |
| Fault Tolerance   |   Less efficient (concatenation)  |   More efficient (parallel X/Z checks) |
| Use Case  |   Proof-of-concept    |   Practical implementations |